#### 제목 : 반도체 제조 클러스터 툴의 스케줄링 이론 및 방법 검토

1. INTRODUCTION
   - 대부분의 처리도구들은 처리량을 최대화하기 위해 웨이퍼를 카세트(25개)의 배치사이즈로 처리한다.
   - 그러나 웨이퍼 크기증가, 품질요구수준이 더 엄격해지고, 회루수축 때문에, 제조과정이 더 복잡해지고있다.
   - 그래서 채택한게 SWP(Single-Wafer Processing) : 웨이퍼를 챔버 안에서 하나하나씩 처리하는 기술
   - 그런데 SWP단점이 챔버간 이송작업이 너무많다.
   - 그래서 여러 챔버들을 하나의 tool로 결합하고자 한다.
   - 그것이 바로 Cluster tools(track equipment) : wafer handling robot
   <br><br>
   - 이 논문에서 소개할 내용
   - 1. tool architecture
   - 2. operation issues
   - 3. scheduling requirements
   - 4. recent progress in tool science
   - 5. engineering for scheduling
   - 6. control of cluster tools
   - 그 외에도 schedule quality의 개념, minimizing the tool cycle time, controlling wafer delays, workload balancing의 개념, dispatching rules for cluster tool scheduling등을 소개하면서
   - 최종적으로, 스케줄링과 제어를 위한 제어 소프트웨어 아키텍쳐를 설명

2. CLUSTER TOOL ARCHITECTURES, OPERATION, AND SCHEDULING REQUIREMENTS
    <br>
    <br>
    - 2.1. Basic Cluster Tool Architecture and Operation
        - 보통 cluster tool은 방사형 모양이고, 6개 정도의 챔버, single wafer-handling robot 1개를 가진다.
        - 챔버가 추가되거나 빠질수도 있는데, 그러면 길고 non-uniform한 이동시간이 유발된다. 로봇 또한 팔을 하나 가질수도, 두 개(dual-arm)를 가질수도 있다. 단, 팔이 두 개일 때는, 움직이는 공간을 반씩 나눠 움직이는 각도를 고정해놓는다.
        - Dual-armed tool은 더 큰 처리량을 보이지만 그만큼 비용도 커진다.
        - 보통 cassette는 loadlock에 저장됨, 앞으로 편의를 위해, loadlock에 저장된 cassette를 그냥 loadlock이라 하자.
        #### **질문 : loadlock은 그냥 처리중인 생산품을 저장해놓는 역할?**
        - 카세트에 적재된 웨이퍼 loadlock에서 unload -> 챔버1로 load -> 다 되면 unload -> 챔버2로 load -> 그러면서 새 웨이퍼 챔버1로 load -> 하나하나씩 반복
        - 처리시간이 긴 처리단계는 2개 이상의 병렬의 챔버가 필요할수도 있다.
        - 따라서 챔버를 통과하는 일반적인 웨이퍼 흐름 패턴은 직렬 병렬이다.
        - cassette에 담긴 마지막 웨이퍼가 챔버에 실리면, 그 옆의 loadlock에 load된 wafer를 꺼내서 쓴다. 그러면서 계속 loadlock에 cassette를 계속 보충해준다.
        #### **질문 : 방금 위에 적은 내용 제대로 이해한거 맞나??**
        - aligner : 웨이퍼가 arm에 잘 위치할수있게 정렬해주는 역할(레이저 빔으로 정렬해줌)
        - 기계적인 문제로 웨이퍼가 loadlock에서 꺼내질때, arm에 잘 위치하지 못할수도 있으니까 aligner가 챔버의 슬롯에 잘 들어갈 수 있도록 위치를 재정렬해줌
        - align 소요시간 : 5~10초
        - 쿨러 모듈도 있을수 있음(웨이퍼가 뜨거우면 loadlock에 있는 다른 웨이퍼에도 영향을 줄 수 있기 떄문)
        - 공간 제약으로 인해 챔버간 버퍼는 없음
        - 간혹 있는 경우도 있는데 이러면 로봇 작업량 증가, 웨이퍼가 과도하게 냉각됨
    <br>
    <br>
    - 2.2 Advanced Cluster Tool Architectures and Operation
        - 챔버는 load된 웨이퍼를 처리하기위해 진공, 환기 사이클을 반복한다.
        - 몇몇 tool들은 진공버퍼를 챔버와 loadlock사이에 둬서 챔버의 전체영역을 진공상태로 유지한다. 진공, 환기사이클을 만들지 않기 위해.
        - 몇몇 최신 tool들은 챔버에 다중 웨이퍼슬롯을 가진다. 이것은 여러개의 웨이퍼를 함께 처리하기 때문에 SWP보다 생산량을 더 증가시킨다. 그러나 그만큼 스케줄 복잡도도 상당히 증가한다.
        - 다중 핸들링 로봇과 SWP 챔버의 통합 시스템을 track tool 또는 track system이라고 한다.
        #### **질문 : 포토리소그래피 공정은 광민성 화학 물질로 코팅된 웨이퍼를 스테퍼에 공급하고 스테퍼에서 회로 패턴 사진 이미지에 노출되어 형성된 웨이퍼의 회로 패턴을 개발하는 트랙 시스템을 사용한다.???**
        - 포토리소그래피 공정, Process module, 자동 습식 스테이션 도 여러개의 module, robot들로 시스템이 구성되어있다.
        - 최근에는 웨이퍼의 테스트디바이스를 위한 EDS프로세서가 track system의 형식으로 구성되어있다.
        - rail-guided vehicle(RGV)로 불리는 레일에서 움직이는 로봇이 load,unload작업을 하고 다중 병렬로 연결된 여러 테스트 툴들이 있다.
        - RGV의 service zone(움직이는 구간)은 서로 조금씩 곂쳐져있고 공정 중간의 버퍼가 없어서 로봇 충돌이나 deadlock이 발생할 수도 있다.
        - 몇몇 cluster tool은 딜레이를 줄이기위해 중간버퍼를 연결해 작업 끝난 웨이퍼를 바로 다음 tool로 넘기는 것도 있다.
        - radial-tpye의 cluster tool들은 오직 로테이션하게 움직이기도 하고, wet station의 경우 수평적으로 움직이기만 하고, track equipment는 수직,수평적으로 움직이기도 한다.
        - 이렇게 각각 다른 로봇 움직임 패턴은 각기 다른 스케줄링 결과를 만들어낸다.
    <br>
    <br>
    - 2.3 Tool Scheduling Requirements
        - 웨이퍼는 대체로 처리단계가 직렬로 되어있다.
        - 처리단계의 챔버가 여러개면 그중의 하나가 랜덤으로 처리단계에 쓰일수 있다
        - 그것의 대안으로, 병렬로 된 챔버가 돌아가면서 쓰일수도 있다.
        - 그래서 병렬챔버로된 tool의 처리단계에서 균일한 처리시간이 소요된다고 판단된다.
        - 그리고 방사형 cluster tool에서 챔버간 이동시간은 서로 크게 차이나지않아서 서로 같으면서도 작다고 판단된다.
        - 그러나 수직적, 수평적으로 구성된 챔버들(wet station, track equipment 같은)은 챔버나 bath간 이동시간이 다르다고 판단된다.
        - cluster tool의 웨이퍼 흐름은 기존의 흐름 라인 또는 조립 라인과 유사하게 보일 수 있다. 그러나 챔버 간에는 중간 버퍼가 없으며 로봇 암의 가용성에 따라 이동이 제한된다.
        - 어떤 프로세스의 경우, 웨이퍼가 한번 거쳐잔 프로세스에 다시 올 때도 있다.
        - 그렇게 됐을때, 돌아오는 웨이퍼와 신규로 진입하려는 웨이퍼의 작업순서는 고려되어야한다. 그것 도한 챔버가 병렬적인지도 고려했을때, 다른 전략이 사용된다.
        - 몇몇 프로세스에서는, 웨이퍼를 몇개 작업할 때 마다 챔버를 세척해야하거나 특정량 이상의 오염이 감지될 때 챔버를 세척하기도 한다. 또는 그냥 아예 주기적으로 하기도 한다.
        - 챔버 세척(Chamber cleaning)또한 스케줄링 되어야하는 작업이다.
        - wafer delay(wafer residency time) : 작업이 끝났는데도 챔버에 남아있는 웨이퍼의 대기시간
        - wafer delay는 챔버 내의 잔류 가스 및 열로 인해 표면 품질 문제가 발생할 수 있다.
        - wafer delay가 상한선을 넘으면 그 웨이퍼는 폐기된다.
        - 그래서 wafer delay는 바람직하지않고, 줄여지거나, 제거되거나, 일정하도록 통제되어야 한다.
        - Process time은 스케줄링 모델에서 결정론적인 요소이다.
        - 스케줄링 모델이 무작위 변동 없이 결정론적 프로세스와 작업 시간을 가정하는지 또는 예외적인 지연 또는 확률적 시간을 고려하는지 여부를 결정해야 한다.
        - 모델링과 분석에서의 복잡도, 어려움, 무작위성의 정도 또한 고려되어야한다.
        - 결정론적 모델은 더 간단한 분석과 더 나은 결론을 주고,이를 통해 우리는 실질적인 구현을 위한 랜덤한변동을 다루는 방법을 발달시킬수 있다.
        - 그러나 종종 현실에 의해 분석이나 스케줄링 최적화는 제한되고 종종 모델 단순화가 요구되기도 한다.
        - 시뮬레이션은 현실적인 모델을 제공한다, 그러나 인과관계, 매게관계를 식별하는데 한계를 가진다.
        - 통합된 tool은 대게 중간버퍼를 제한한다.
        - 그러므로 blocking, waiting, 심지어는 deadlock까지 발생할수있다.
        - 따라서 재진입, wafer delay, cleaning cycle, 그리고 불확실성은 스케줄링 복잡도를 크게 증가시킬수 있다.
        - 똑똑한 스케줄링과 조절에의한 tool 생산력은 팹의 생산력 극대화에 중대하다.
        

 3. SCHEDULING STRATEGIES FOR CLUSTER TOOLS
    - 클러스터 툴을 위한 대안적인 스케줄링 전략이 있다.
    1. dispatching rule : tool의 상태에 따라 다음 로봇의 task를 결정한다. 그리고 그것은 동적이고, 실시간으로 이루어진다.
    - 그러나 이 규칙을 최적화하기는 힘들다. 우리는 컴퓨터에 의해 휴리스틱하게 고안된 dispatching rule의 성능만 비교할 수 있다.
    2. 미리 결정되는 스케쥴, 적절한 스케줄링 모델만 정의된다면 성능을 최적화 할 수 있다.
    - tool의 상황에 상당한 변화가 일어난다면, 리스케줄이 발생할 수 도 있다.
    - Cyclic scheduling은 각 로봇과 챔버가 같은 work cycle을 반복하도록 한다.
    - 일단 robot task cycle이 정해지면, 모든 work cycle도 정해진다.
    - 대부분의 학술연구는 cluster tool schedulling에서 cyclic schedulling을 고려한다.
    - cyclic schedulling 장점 : 감소된 schedulling복잡도, 예측가능한 행동, 생산성 향상, 일정하고 주기적인 패턴 등등
    - cyclic schedulling에서 일의 타이밍은 미리 정의된 work cycle동안 연관된 일의 발생에 의해서 제어된다.


4. CYCLIC SCHEDULING
    - 4.1 Modeling Cluster Tool Behavior
        - cluster tool의 작동행동은 Petri net을 통해 잘 모델링 된다.
        - Petri net : 이산적인 사건시스템을 다루는 그래피컬하고 수학적인 모델링 프레임워크
        - Transition, place, arc, token로 주로 표현된다.
        - 주기적 스케줄링에 의해 동일한 작업 주기를 반복하는 cluster tool은 TEG(timed event graph)로 주로 표현된다.
        - TEG는 하나의 input과 output 전환만을 가진다.
        - 일단 로봇의 task sequence가 주어지면, TEG가 정의된다.
        - TEG가 만들어지면, tool cycle time, optimal robot task sequence, wafer delay, optimal timing schedule이 정의된다.
        - #### **질문 tool cycle time은 TEG 모델에서 회로 비의 최대값으로, 회로의 총 시간과 회로의 토큰의 수를 합한 값이다.(???)**
        - Petri net 모델에는 재진입 웨이퍼 플로우, 중간버퍼를 가진 tool, wet station, 멀티슬롯 챔버를 가진 tool, cleaning cycle을 가진 tool 등이 있다.

![image](https://ifh.cc/g/Y6g4CP.png)
<br>
Figure 4: A timed event graph model for a dual-armed cluster tool

- 
    - 4.2 Schedule Quality
        - 주어진 순환 시퀀스를 갖는 클러스터 툴의 경우, 서로 다른 종류의 스케줄이 있을 수 있으며, 각각의 스케줄은 TEG 모델의 점화 스케줄에 대응된다.
        - 주기적인 스케줄은 매 $d$ work cycle만큼 같은 타이밍 패턴을 반복한다.
        - $d=1$일때, 스케줄은 안정적이라고 불린다.
        - 안정적인 스케줄에서, wafer delay같은 모든 task delay는 일정하다.
        - $d$-periodic한 스케줄에서, wafer delay는 d가지의 다른 value를 가진다, 반면에, 평균은 안정적인 스케줄의 평균과 같다.
        - period $d$는 TEG를 통해 결정된다.
        - 이전 작업이 완료되는 즉시 각 작업을 시작하는 일정을 '$earliest$'라고 합니다.
        - 각 전환이 활성화되는 즉시 실행(fire)되는 TEG 모델의 초기 실행 규칙에 의해 가장 빠른 일정을 생성할 수 있다.
        - TEG모델의 ealist firing rule은 실시간의 스케줄러나 컨트롤러로 이용된다.
        - 달리말해, 최초의 스타팅 스케줄은 미리 생성되거나 저장될 필요가 없다.
        - 따라서, 적절한 이벤트(예: 작업 완료)가 발생할 때 작업을 시작하는 이벤트 기반 제어에 의해 가장 빠른 일정을 구현할 수 있다.
        - 그러므로, 이벤트 기반의 최초의 스타팅 스케줄은 장점을 가진다.
        - 첫번째로, 메세지 시퀀스 변화로 인한 잠재적인 논리적 에러가 예방된다.
        - tool이 미리 정해진 스케줄로 통제되고 있을때, 통신이나 컴퓨팅지연이 메세지 시퀀스의 변화를 일으킬수 있으면 그리고 그래서 중대한 논리적 오류를 야기할 수도 있다.
        - 예를 들어 로봇이 챔버에서 처리가 완료되지 않아 웨이퍼 슬롯이 여전히 닫혀 있을 때 챔버에서 웨이퍼를 언로드하려고 할 수 있다.
        - 두번째로, ealiest 스케줄은 평균 tool cyle time을 최소화한다.
        - 그러므로, 가장 바람직한 스케줄은 $steady$ $and$ $ealist$ $starting$ $schedule(SESS)$ : 안정적이가 가장 빠른 시작일정 이다.
        - SESS는 max-plus 대수 또는 일종의 longest path 알고리즘을 사용하여 미리 계산될 수 있으며 TEG 모델에 기반한 이벤트 기반 컨트롤러에 의해 구현된다.
        - ## **질문 : 그래서 SESS가 정확히 무엇일까?**

![image](https://ifh.cc/g/C8tFgd.png)
<br>
Figure 5: Examples of schedules.

-   - 4.3 Cycle Time Optimization
        - backward sequence : single - armed tool이 주로 행하는 sequence 방식
        - swap sequence : dual - armed tool
        - backward sequence : 웨이퍼를 마지막 처리단계(n)의 챔버에서 빼준 후, 그것을 로드락으로 옮김, 그러고나서 n-1처리단계의 웨이퍼를 챔버에서 빼서 n처리단계의 챔버로 넣어준다.
        - swap sequence : 로봇은 두 개의 암을 사용하여 프로세스 단계 i의 챔버에서 처리된 웨이퍼와 로봇 암의 웨이퍼를 교환합니다. 그런 다음, 로봇 암의 웨이퍼가 다음에 방문해야 하는 공정 단계 i+1의 챔버에 대해 이 스왑 작업이 수행됩니다.
        - #### **질문 챔버의 수, 팔의 수 등 여러 상황에 따라 최적은 다르지만, 핵심 아이디어는 swap sequence에 대한 tool cycle time은 챔버 작업 사이클 또는 로봇 작업 사이클의 작업량과 동일하다는 것을 보여준다.**
        - ####  **workload?, tool cycle time?**
        - cyclic operation의 클러스터 툴 스케줄링 문제는 MIP문제로 모델링 할 수 있다.
        - 포인트는 먼저 TEG에대한 안정적인 일정, 작업의 타이밍을 결정하기 위해 LP모델을 개발한 뒤, 작업 시퀀스를 정하기위해 그걸 MIP모델로 확장하는 것이다.
        - swap operation자체는 두 암의 작동을 제한한다. 최근엔 swap-restriction없는 모델이 개발되기도.

-   - 4.4 Controlling Wafer Delays
        - 4.4.1 Schedulability Analysis
            - Schedulability : 클러스터 툴 또는 로봇 셀의 스케줄링 가능성, 즉 실현 가능한 스케줄의 존재
            - 클러스터 도구 또는 로봇 셀의 스케줄링 가능성, 즉 실현 가능한 스케줄의 존재에 대한 연구가 있었다. 스케줄러빌리티, 즉 TEG, NEG의 확장 버전의 회로와 실행 가능한 SESS를 계산하는 절차에 기초한 실행 가능한 스케줄의 존재에 대한 필요충분조건이 있다. 실제로 스케줄러빌리티는 관련 LP에 실현 가능한 솔루션의 존재에 의해 검증될 수 있다. 그러나 필요하고 충분한 조건은 시간 제약이 위반되는 이유를 식별하고, 종종 프로세스 시간, 로봇 작업 시간 및 각 프로세스 단계의 병렬 챔버 수와 같은 스케줄링 매개 변수를 기반으로 폐쇄형 스케줄러빌리티 조건을 제공한다. 예를 들어 이중 무장 클러스터 도구의 프로세스 시간 범위는 'u(n+1)v+ns+rmin1inti+di+smi'와 'max1infti+smig min1infti+di+smig'이다. NEG 기반의 스케줄링 방법은 시간 제약이 있는 주기적 운영이 있는 다른 클러스터 도구 또는 자동화된 시스템에 사용될 수 있다. 프로세스 시간 및 로봇 작업 시간에 시간 변동이 있더라도 이들의 다양한 범위가 유한한 간격으로 식별되면 스케줄링 가능성을 효율적으로 검증할 수 있다. 이는 시간 변동에 대비하여 공구 일람표를 안전하게 유지하는 데 유용합니다.
            - ### **이 단락은 내용자체가 이해가 안됨,,schedulability의 뜻은 저게 맞는지, schedulability의 존재유무는 중요한지 등등 일단 번역본 복붙해놨음, 계속 읽어보자**

-   - 
        - 4.4.2 Wafer Delay Identification(웨이퍼 지연 식별)
            - ### **질문 : periodic schedule이란 일정한 루틴을 반복하는 schedule인건가?**
            - 초기 타이밍이 적절하게 제어되지 않거나 SESS가 중단될 경우, 가장 이른 일정은 TEG로부터 결정되는 주기적 스케줄로 수렴된다.
            - 따라서 웨이퍼 지연은 SESS의 상수 값보다 훨씬 클 수 있다.
            - 주어진 웨이퍼 지연 제약의 경우, 스케줄러빌리티 조건, 즉 실현 가능한 SESS가 존재하더라도 주기적인 스케줄은 한계를 초과하는 웨이퍼 지연을 가질 수 있다.
            - 따라서 웨이퍼 지연이 변동하는 이러한 주기적인 스케줄이 웨이퍼 지연 제약 조건을 충족할 수 있는지 우려됩니다.
            - 시간 중단에 대한 최악의 웨이퍼 지연은 주기적인 일정에 대한 토큰 지연을 식별하여 계산할 수 있다.


-   - 4.5 Regulating Wafer Delays: Schedule Stability
        - SESS에 의해 클러스터 툴이 가동될때, wafer delay등 대부분의 지표는 이론상 일정하만, 현장에선 여러 변수로 그럴수 없을때도 많다.
        - 그럴 경우 방해받은 스케줄을 빠르게 회복시켜주는 방법이 있다.
        - Schedule Stability condition : 초기 점화 일정이 중단 크기에 관계없이 원래 SESS로 수렴되는 일정 안정성 조건, 일부 선택된 작업에 적절한 지연을 추가하여 이러한 안정성을 시행

-   - 4.6 Reducing Wafer Delays: Workload Balancing for Tools
        - process step : 해당 step의 모든 작업의 process time의 합
        - process step간의 작업량의 불균형 바틀넥 이전의 작업대기나 WIP를 발생
        - 하지만 클러스터 툴같은 자동제조시스템에서는 '작업량'이라는걸 정의하기 힘들어서 작업량의 정의를 확장하고자 한다.
        - 클러스터 툴에서의 작업량 : 자원에 대한 일반화된 작업량을 자원에 대한 작업주기에 해당하는 TEG의 회로비
        - $p_i$ : wafer processing
        - $l$ : loading task
        - $u$ : unloading task
        - $r$ : robot move
